В текущем задании вам предстоит построить рекомендательную систему постов в социальной сети. В качестве базовых сырых данных вы будете использовать подготовленные заранее командой курса таблицы.
С точки зрения разработки вам будет необходимо реализовать сервис, который будет для каждого юзера в любой момент времени возвращать посты, которые пользователю покажут в его ленте соцсети.
Метрика для проверки качества модели - hitrate@5.

Описание данных
 
Таблица user_data
Cодержит информацию о всех пользователях соц.сети.

age:	    Возраст пользователя (в профиле)
city:	    Город пользователя (в профиле)
country:	Страна пользователя (в профиле)
exp_group:	Экспериментальная группа: некоторая зашифрованная категория
gender:	    Пол пользователя
id:	        Уникальный идентификатор пользователя
os:	        Операционная система устройства, с которого происходит пользование соц.сетью
source:	    Пришел ли пользователь в приложение с органического трафика или с рекламы

Таблица post_text_df
Содержит информацию о постах и уникальный ID каждой единицы с соответствующим ей текстом и топиком.

id:	    Уникальный идентификатор поста
text:	Текстовое содержание поста
topic:	Основная тематика

Таблица feed_data
Содержит историю о просмотренных постах для каждого юзера в изучаемый период.

timestamp:	Время, когда был произведен просмотр
user_id:	id пользователя, который совершил просмотр
post_id:	id просмотренного поста
action:	    Тип действия: просмотр или лайк
target: 	1 у просмотров, если почти сразу после просмотра был совершен лайк, иначе 0. У действий like пропущенное значение.


Оптимизация HitRate@5
Для начала напомним, как устроена метрика. Она принимает значение 1, если среди предложенных 5 рекомендаций хотя бы 1 получила в итоге like от пользователя. Даже если все 5 предложенных постов в итоге будут оценены пользователем, все равно hitrate будет равен 1. Метрика бинарная! В противном случае, если ни один из предложенных постов не был оценен пользователем, hitrate  принимает значение 0. Эту метрику мы хотим максимизировать.

Валидация
В рамках задачи вам необходимо будет предсказывать посты для тех же юзеров, что и есть в текущей выборке, но в будущем. Следовательно, валидацию стоит организовать таким же образом. 

In [1]:
# Имортируем библиотеки

import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [2]:
# Загружаем данные и сохраняем их на всякий случай на диске

user_data = pd.read_sql(
    """SELECT * FROM public.user_data""",
    con="postgresql://robot-secret"
        "postgres.lab.karpov.courses:secret"
)

user_data.head()

,user_id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


In [3]:
user_data.shape

(163205, 8)

In [4]:
user_data['user_id'].nunique()

163205

In [5]:
user_data.to_csv('user_data.csv')

In [6]:
post_df = pd.read_sql(
    """SELECT * FROM public.post_text_df""",
    con="postgresql://robot-secret"
        "postgres.lab.karpov.courses:secret"
)

post_df.head()

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business


In [7]:
post_df.to_csv('post_df.csv')

In [8]:
feed_data = pd.read_sql(
    """SELECT * FROM public.feed_data limit 100000""",
    con="postgresql://robot-secret"
        "postgres.lab.karpov.courses:secret"
)

feed_data.head()

,timestamp,user_id,post_id,action,target
0,2021-11-10 20:22:48,145195,4738,view,0
1,2021-11-13 12:45:48,145195,2835,view,1
2,2021-11-13 12:48:30,145195,2835,like,0
3,2021-11-13 12:48:32,145195,4374,view,0
4,2021-11-13 12:51:07,145195,5534,view,0


In [9]:
feed_data.to_csv('feed_data.csv')

In [10]:
# Объединяем таблицу в единый датасет

df = pd.merge(
    feed_data,
    user_data,
    on='user_id',
    how='left'
)

df = pd.merge(
    df,
    post_df,
    on='post_id',
    how='left'
)

df.head()

,timestamp,user_id,post_id,action,target,gender,age,country,city,exp_group,os,source,text,topic
0,2021-11-10 20:22:48,145195,4738,view,0,0,21,Russia,Starokucherbayevo,0,iOS,organic,"As a fan of the old Doctor Who, and after the ...",movie
1,2021-11-13 12:45:48,145195,2835,view,1,0,21,Russia,Starokucherbayevo,0,iOS,organic,I think even those devastated by #LoanChargeSc...,covid
2,2021-11-13 12:48:30,145195,2835,like,0,0,21,Russia,Starokucherbayevo,0,iOS,organic,I think even those devastated by #LoanChargeSc...,covid
3,2021-11-13 12:48:32,145195,4374,view,0,0,21,Russia,Starokucherbayevo,0,iOS,organic,This TVM seems to have polarised opinions amon...,movie
4,2021-11-13 12:51:07,145195,5534,view,0,0,21,Russia,Starokucherbayevo,0,iOS,organic,I havent laughed this much in a long time - or...,movie


In [11]:
df.isna().sum()

timestamp    0
user_id      0
post_id      0
action       0
target       0
gender       0
age          0
country      0
city         0
exp_group    0
os           0
source       0
text         0
topic        0
dtype: int64

In [12]:
# Считаем количество постов с лайками

likes = df.query('action=="like"').groupby(['user_id', 'post_id'], as_index=False)['target'].agg('count')\
        .rename(columns={'target':'like'})

In [13]:
likes.shape

(10040, 3)

In [14]:
# Считаем количество постов с просмотрами, после которых сразу же был поставлен лайк

views = df.query('action=="view"').groupby(['user_id', 'post_id'], as_index=False)['target'].agg('sum')\
        .rename(columns={'target':'lliked_view'})

In [15]:
liked_views = views.merge(likes, on=['user_id', 'post_id'])
liked_views.shape

(10040, 4)

In [16]:
liked_views.head()

,user_id,post_id,lliked_view,like
0,29863,548,1,1
1,29863,1083,1,1
2,29863,1581,1,1
3,29863,4935,1,1
4,29863,5694,1,1


In [17]:
liked_views.lliked_view.value_counts()

1    9962
2      76
3       2
Name: lliked_view, dtype: int64

Поскольку у всех просмотров в данной таблице target>0, то они все лайкнуты сразу после просмотра. 
Также их количество примерно равно количеству постов с like. Однако, из таблицы куда-то потерялись несколько постов с like - возможно, данные по их просмотру "срезались" при выгрузке. Таким образом, можно удалить из таблицы все посты с like, а по оставшимся постам с view посчитать среднее по таргету - так мы получим метрики, как часто пользователь ставит лайки и как часто ставят лайки этому посту.

In [18]:
df = df.query('action=="view"')

In [20]:
user_likes = df.groupby('user_id')['target'].mean() #соотношение лайков и просмотров для каждого пользователя
post_rate = df.groupby('post_id')['target'].mean()  #"рейтинг" каждого поста

In [21]:
# создадим колонки с рейтингом
df['user_rate'] = df['user_id'].map(user_likes)
df['post_rate'] = df['post_id'].map(post_rate)

Поскольку один пользователь может просматривать пост несколько раз, необходимо оставить только по одному просмотру на пост. Если пост лайкнут - значит оставить этот просмотр, если не лайнкут - оставить только один из просмотров.

In [38]:
# сразу создадим дф с признаками пользователей
user_features = user_data.merge(df[['user_id', 'user_rate']].drop_duplicates(), on='user_id', how='left')\
                .fillna(df['user_rate'].mean())

In [40]:
# для каждого поста оставим только минимальное время, которое будем считать временем публикации

post_time = feed_data.groupby('post_id', as_index=False)['timestamp'].min()

# сразу создадим дф с признаками постов
post_features = post_df.merge(df[['post_id', 'post_rate']].drop_duplicates(), on='post_id', how='left')\
                .fillna(df['post_rate'].mean())

post_features = post_features.merge(post_time, on='post_id', how='left')

In [23]:
# создаём датафрейм, в котором будут только лайкнутые посты
liked = df.query('target==1').groupby(['user_id', 'post_id'], as_index=False).size()
liked.shape

(10041, 3)

In [24]:
# создаём датафрейм с просмотрами без лайков
non_liked = df.query('target==0').groupby(['user_id', 'post_id'], as_index=False).size()

# смотрим, сколько раз один и тот же пост может попадаться одному и тому же пользователю
non_liked['size'].unique()

array([1, 2, 3, 4], dtype=int64)

In [25]:
# далее для каждой пары юзер-пост проверяем, есть ли этот пост среди лайков.
# если нет - ставим 0, если есть - 1. Далее отфильтровываем 1.
for user in non_liked['user_id'].unique():
    liked_posts = liked[liked['user_id']==user]['post_id'].to_list()
    for post in non_liked[non_liked['user_id']==user]['post_id'].unique():
        if post not in liked_posts:
            non_liked.loc[(non_liked['user_id']==user)&(non_liked['post_id']==post), 'size'] = 0
        else:
            non_liked.loc[(non_liked['user_id']==user)&(non_liked['post_id']==post), 'size'] = 1

In [26]:
non_liked['size'].unique()

array([0, 1], dtype=int64)

In [27]:
non_liked = non_liked.query('size==0') # оставим только посты без лайков
non_liked.shape

(75686, 3)

Таким образом, у нас есть 2 датафрейма: в одном для каждого пользователя содержатся только уникальные посты с лайками, а в другом - только уникальные посты с просмотрами, без лайков.

In [28]:
user_posts = pd.concat([non_liked, liked]).rename(columns={'size':'target'}) # объединим всё в единый дф
user_posts.head()

,user_id,post_id,target
0,29863,1269,0
1,29863,1297,0
2,29863,1524,0
3,29863,1601,0
4,29863,1806,0


In [41]:
user_features = user_posts.merge(user_feat, on='user_id', how='left') # объединим этот дф с признаками пользователей и постов
user_features = user_features.merge(post_features, on='post_id', how='left')
user_features.head()

,user_id,post_id,target,gender,age,country,city,exp_group,os,source,user_rate,text,topic,post_rate,timestamp
0,29863,1269,0,0,19,Russia,Tambov,1,iOS,ads,0.241379,Blair and Brown criticised by MPs\n\nLabour MP...,politics,0.103448,2021-10-01 21:54:51
1,29863,1297,0,0,19,Russia,Tambov,1,iOS,ads,0.241379,Could rivalry overshadow election?\n\nTony Bla...,politics,0.095238,2021-10-02 14:07:02
2,29863,1524,0,0,19,Russia,Tambov,1,iOS,ads,0.241379,Italy aim to rattle England\n\nItaly coach Joh...,sport,0.142857,2021-11-17 12:30:06
3,29863,1601,0,0,19,Russia,Tambov,1,iOS,ads,0.241379,Isinbayeva claims new world best\n\nPole vault...,sport,0.000000,2021-10-01 20:27:04
4,29863,1806,0,0,19,Russia,Tambov,1,iOS,ads,0.241379,Campbell rescues Arsenal\n\nSol Campbell prove...,sport,0.157895,2021-10-02 11:03:11


In [42]:
user_features.shape

(85727, 15)

In [43]:
user_features.to_csv('user_features.csv')

In [202]:
### Разделим на трейн-тест
user_features = user_features.sort_values(by='timestamp')

train = user_features.iloc[:-20000].copy()
test = user_features.iloc[-20000:].copy()

In [203]:
train_new = train.drop(['user_id', 'post_id',  
                         'exp_group', 'text', 'timestamp'], axis=1) 

test_new = test.drop(['user_id','post_id',  
                        'exp_group', 'text', 'timestamp'], axis=1) 

# удаляем лишние колонки. exp_group будет использована в АВ тестировании, а text - в другой модели с использованием NLP.

In [204]:
train_new.head()

,target,gender,age,country,city,os,source,user_rate,topic,post_rate
24210,0,1,29,Russia,Lipetsk,Android,ads,0.042208,politics,0.086957
54656,0,1,27,Ukraine,Kamianske,iOS,organic,0.234513,politics,0.151515
84051,1,0,27,Russia,Volzhsk,Android,organic,0.056686,tech,0.200000
54420,0,1,30,Russia,Roslavl,Android,organic,0.078049,politics,0.157895
57282,0,0,38,Russia,Moscow,iOS,organic,0.094241,business,0.055556


In [205]:
X_train = train_new.drop('target', axis=1)
X_test = test_new.drop('target', axis=1)

y_train = train['target']
y_test = test['target']

In [206]:
cat_features=['gender', 'country', 'city', 'os', 'source', 'topic'] # обозначаем категориальные колонки

In [ ]:
# Создаём и обучаем модель

from catboost import CatBoostClassifier

catboost = CatBoostClassifier()


catboost.fit(X_train,
             y_train,
             cat_features=cat_features,
             )

In [ ]:
# Для каждой строки в тексте добавляем предсказанную вероятность лайка, а также само предсказание

test_new = user_features.iloc[-20000:].copy()

X_test['prob'] = catboost.predict_proba(X_test)[:, 1]
X_test['pred'] = catboost.predict(X_test)
X_test['target'] = y_test
X_test['user_id'] = test_new['user_id']
X_test['post_id'] = test_new['post_id']

In [209]:
# Считаем целевую метрику

hitrate5 = []

for user in X_test['user_id'].unique():
    part = X_test[X_test['user_id']==user]
    if sum(part['pred']==1)==0:
        continue # пропускаем пользователя, если для него нет рекомендаций
        
    # далее для каждого пользователя отбираем посты, для которых предсказаны лайки, и сортируем их по вероятности
    elif sum(part[part['pred']==1].sort_values(by='prob', ascending=False)[:5]['target']==1)>0:
        hitrate5.append(1)
    else:
        hitrate5.append(0)
        
print(f"Среднее HitRate@5 по пользователям из теста: {round(np.mean(hitrate5), 2)}")

Среднее HitRate@5 по пользователям из теста: 0.82


In [210]:
# Сохраняем модель
catboost.save_model('catboost_model',
                           format="cbm")